In [1]:
# scripts/fix_doyle_metrics.py
from core.libs import pd, text
from core.db import get_engine

In [3]:
def backup_table(tabla_original, backup_schema="backups"):
    """
    Hace un respaldo rápido de la tabla SQL usando CREATE TABLE ... AS SELECT.
    El backup se guarda en el esquema 'backups' (créalo si no existe).
    """
    engine = get_engine()
    backup_table = f"{backup_schema}.{tabla_original.split('.')[-1]}_bkp"
    with engine.connect() as conn:
        # Crear esquema de backups si no existe
        conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {backup_schema}"))
        # Borrar backup anterior si existe
        conn.execute(text(f"DROP TABLE IF EXISTS {backup_table}"))
        # Clonar tabla
        conn.execute(text(f"CREATE TABLE {backup_table} AS TABLE {tabla_original}"))
        print(f"✅ Backup creado: {backup_table}")

# Ejemplo de uso:
backup_table("masterdatabase.inventory_metrics")
backup_table("masterdatabase.inventory_metrics_current")

💻 Conectado a la base de datos helloworldtree
✅ Backup creado: backups.inventory_metrics_bkp
💻 Conectado a la base de datos helloworldtree
✅ Backup creado: backups.inventory_metrics_current_bkp


In [4]:

def fix_doyle_and_count(table_name="masterdatabase.inventory_metrics", groupby_fields=["contractcode"], commit_changes=False):
    """
    Recalcula doyle_bf solo para árboles con DBH >= 8 y cuenta cuántos cumplen ese criterio por grupo.
    """
    engine = get_engine()
    print(f"📥 Leyendo tabla: {table_name}")
    df = pd.read_sql(f"SELECT * FROM {table_name}", engine)

    # Asegura los nombres correctos de las columnas
    dbh_col = "dbh_in"
    tht_col = "tht_ft"
    doyle_col = "doyle_bf"

    # Convierte a numérico
    df[dbh_col] = pd.to_numeric(df[dbh_col], errors='coerce')
    df[tht_col] = pd.to_numeric(df[tht_col], errors='coerce')

    # Aplica la fórmula de Doyle solo si dbh >= 8
    df[doyle_col] = ((df[dbh_col] - 4) ** 2) * (df[tht_col] / 16)
    df[doyle_col] = df[doyle_col].where(df[dbh_col] >= 8, other=pd.NA)

    # Conteo de árboles válidos por grupo (ejemplo: por contrato)
    filter_mask = df[dbh_col] >= 8
    group_counts = df[filter_mask].groupby(groupby_fields).size().reset_index(name="count_doyle")
    print("\n🔢 Conteo de árboles válidos por grupo:\n")
    print(group_counts)

    # Merge para tener count_doyle en el DataFrame principal (opcional)
    if "count_doyle" not in df.columns:
        df = df.merge(group_counts, on=groupby_fields, how="left")

    # Opcional: guarda como copia o sobrescribe
    if commit_changes:
        print(f"💾 Sobrescribiendo tabla {table_name} (haz respaldo primero si es necesario)...")
        df.to_sql(table_name, engine, if_exists="replace", index=False)
    else:
        out_name = table_name + "_fix"
        print(f"💾 Guardando copia en {out_name}")
        df.to_sql(out_name, engine, if_exists="replace", index=False)

    print("✅ Doyle corregido y conteo agregado.")

if __name__ == "__main__":
    # Puedes cambiar los argumentos aquí:
    fix_doyle_and_count(
        table_name="masterdatabase.inventory_metrics_current",  # o "masterdatabase.inventory_metrics"
        groupby_fields=["contractcode"],  # Puedes agregar más campos, ej. ["contractcode", "year"]
        commit_changes=False  # True para sobrescribir la tabla original
    )


💻 Conectado a la base de datos helloworldtree
📥 Leyendo tabla: masterdatabase.inventory_metrics_current


KeyError: 'dbh_in'